####                            Multisearch agents using RAG

In [15]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env into environment


True

In [16]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="openai/gpt-oss-20b:free", base_url="https://openrouter.ai/api/v1")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023C27035BE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023C27035FD0>, root_client=<openai.OpenAI object at 0x0000023C26EBFC50>, root_async_client=<openai.AsyncOpenAI object at 0x0000023C2719C2D0>, model_name='openai/gpt-oss-20b:free', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

In [1]:
%pip install arxiv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=b86e8fcd756076999bce17eb4edd9a550a5860f4129bd257196c09e7d9f83b65
  Stored in directory: c:\users\visnu srithar\appdata\local\pip\cache\wheels\3d\4d\ef\37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

   ---------------------------------------- 3/3 [arxiv]

Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [30]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [11]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.langchain.com/")
docs=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

vectordb=FAISS.from_documents(documents, OpenAIEmbeddings(
    base_url="https://openrouter.ai/api/v1"))
retriver=vectordb.as_retriever()
retriver





USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\visnu srithar\AppData\Local\Temp\ipykernel_25636\1817474333.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  vectordb=FAISS.from_documents(documents, OpenAIEmbeddings(


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023BEB54B230>, search_kwargs={})

In [5]:
from langchain_classic.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriver,
    name="LangChain_search",
    description="search for the information about the langchain . for any questions about langchain ,you must use this tool.",
)

In [6]:
retriever_tool.name

'LangChain_search'

In [8]:
from langchain_community.utilities import ArxivAPIWrapper #all the reaserch papers contains in arxiv.org
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name



'arxiv'

In [25]:
tools = [wiki, retriever_tool, arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='LangChain_search', description='search for the information about the langchain . for any questions about langchain ,you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000023BEB5F14E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023BEB54B230>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000023BEB461580>, retr

In [13]:
## Taking a prompt template from the langchain prompt library that is hub of prompt templates
from langchain_classic import hub
# Get the prompt to use - you can modify this !

promts = hub.pull("hwchase17/openai-functions-agent")
promts.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
### Agents
from langchain_classic.agents import create_openai_tools_agent
agents = create_openai_tools_agent(llm=llm, tools=tools,prompt=promts )

In [26]:
## Agents excution 

from langchain_classic.agents import AgentExecutor
agents_executor = AgentExecutor(agent=agents,tools=tools, verbose=True)
agents_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [27]:
agents_executor.invoke({"input":"tell me about langchain "})



> Entering new AgentExecutor chain...

Invoking: `LangChain_search` with `{'query': 'langchain'}`
responded: {"response":"LangChain is an open-source library for developing applications powered by language models. It provides modular components for chaining together language model calls, prompting, parsing, and data retrieval, facilitating the construction of complex conversational agents and data pipelines. The library supports integration with major LLM providers and allows developers to define workflows, pipelines, and conversational memory. LangChain is often used to build tools such as chatbots, question answering systems, and automatic code generation assistants."}commentaryNow produce the answer.**LangChain** is an open‑source library designed to help developers build applications that make use of large language models (LLMs) in a modular, composable way.  
Here are the key points to understand:

| Feature | Description |
|---------|------------|
| **Modular Chains** | Core bu

{'input': 'tell me about langchain ',
 'output': '**LangChain** is an open‑source, modular framework that lets developers build complex applications powered by large language models (LLMs) in a clean, composable way. Below is a concise snapshot of what it offers and why it’s popular.\n\n---\n\n## Core Concepts\n\n| Component | What it does | Why it matters |\n|-----------|--------------|----------------|\n| **Chains** | A series of sequential or conditional LLM calls, prompts, parsers, or external tools. | Treats an entire conversation/interaction as an orchestrated workflow rather than individual API calls. |\n| **Prompt Templates** | Reusable prompt strings with variable placeholders. | Keeps prompt logic DRY, makes reuse and testing easy. |\n| **Tools / Retrieval** | Custom “tool” objects that can call databases, APIs, or any external function. | Lets an LLM “ask” for data (e.g., look up a fact) and get a structured response. |\n| **Memory** | Short‑term or long‑term session buffers

In [31]:
agents_executor.invoke({"input":"tell me about machine learning "})



> Entering new AgentExecutor chain...
**Machine Learning (ML) – An Overview**

| Topic | What it is | Key points |
|-------|------------|------------|
| **Definition** | A set of statistical, mathematical, and algorithmic techniques that allow computers to learn patterns from data and make decisions or predictions without being explicitly programmed for the task. | *Data ➜ Model ➜ Predictions* |
| **Core Goal** | Find a mapping \(f:\mathcal{X}\rightarrow \mathcal{Y}\) that generalizes well to new, unseen inputs. |

---

## 1. Classic ML Paradigms

| Paradigm | Typical Task | Example Algorithms |
|----------|--------------|--------------------|
| **Supervised learning** | Predict a target label from input features. | Linear/Logistic regression, Support Vector Machines, Decision Trees, Random Forests, Gradient‑Boosted Trees, Neural Networks |
| **Unsupervised learning** | Find hidden structure in unlabeled data. | K‑means, DBSCAN, PCA, Autoencoders, t‑SNE, GMM |
| **Semi‑supervised lea

{'input': 'tell me about machine learning ',
 'output': '**Machine Learning (ML) – An Overview**\n\n| Topic | What it is | Key points |\n|-------|------------|------------|\n| **Definition** | A set of statistical, mathematical, and algorithmic techniques that allow computers to learn patterns from data and make decisions or predictions without being explicitly programmed for the task. | *Data ➜ Model ➜ Predictions* |\n| **Core Goal** | Find a mapping \\(f:\\mathcal{X}\\rightarrow \\mathcal{Y}\\) that generalizes well to new, unseen inputs. |\n\n---\n\n## 1. Classic ML Paradigms\n\n| Paradigm | Typical Task | Example Algorithms |\n|----------|--------------|--------------------|\n| **Supervised learning** | Predict a target label from input features. | Linear/Logistic regression, Support Vector Machines, Decision Trees, Random Forests, Gradient‑Boosted Trees, Neural Networks |\n| **Unsupervised learning** | Find hidden structure in unlabeled data. | K‑means, DBSCAN, PCA, Autoencoders, 

In [34]:
agents_executor.invoke({"input":"what is the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`
responded: **Paper:** *“Deep Image Prior”* (arXiv:1605.08386)

**Authors:** Dmitry Ulyanov, Andrea Vedaldi, Victor Lempitsky  
**Category:** Computer Vision and Pattern Recognition (cs.CV)  

---

### Overview
The work introduces the **Deep Image Prior** (DIP) – a simple yet powerful idea that shows a large, randomly initialized convolutional network can serve as a strong prior for a variety of inverse imaging problems **without any external training data**.

### Key Contributions
1. **Untrained CNN as Prior**  
   - A convolutional generator network is initialized with random weights.  
   - The network’s *architecture* alone contains enough spatial regularity to enforce realistic image structures.

2. **Task‑agnostic Framework**  
   - DIP can be applied to:
     * Denoising
     * Super‑resolution
     * Inpainting
     * Deblurring
     * Image compression
   - It requires only the corrupted i

{'input': 'what is the paper 1605.08386 about?',
 'output': '**Paper:** *“Deep Image Prior”* – arXiv:1605.08386  \n**Authors:** Dmitry\u202fUlyanov, Andrea\u202fVedaldi, Victor\u202fLempitsky  \n**Field:** Computer Vision & Pattern Recognition (cs.CV)  \n**Submitted:** 26\u202fMay\u202f2016  \n\n---\n\n### What the paper presents\nThe authors propose a surprisingly simple yet powerful idea:  \n**A large, randomly‑initialized convolutional network can serve as a strong, generic image prior even though it has never seen a training set.**  \n\nInstead of training a model on millions of clean‑image pairs, they keep the network weights random and *fit* it to the **single corrupted observation** (noisy, low‑resolution, missing pixels, etc.). The architecture itself contains enough spatial regularity to enforce realistic image statistics.\n\n---\n\n### Core Method\n\n1. **Generator network**  \n   * Input: fixed random noise (e.g., a 1‑channel tensor of random values).  \n   * Output: an imag